# Affinify: Protein-Ligand Binding Affinity Analysis

This notebook demonstrates the data analysis and model development process for the Affinify project.

## Overview
- Load and explore sample binding affinity data
- Perform molecular descriptor analysis
- Train and evaluate machine learning models
- Visualize results and predictions

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src to path
sys.path.append('../src')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🧬 Affinify Analysis Notebook")
print("Libraries imported successfully!")

## Data Loading and Exploration

In [ ]:
# Load sample data
try:
    from data_processing.data_collector import DataCollector
    
    # Create sample dataset
    collector = DataCollector()
    df = collector.create_sample_dataset()
    
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Display first few rows
    df.head()
    
except ImportError as e:
    print(f"Import error: {e}")
    print("Creating placeholder data...")
    
    # Create placeholder data
    df = pd.DataFrame({
        'protein_name': ['Protein A', 'Protein B', 'Protein C'] * 10,
        'ligand_smiles': ['CCO', 'C1CCCCC1', 'c1ccccc1'] * 10,
        'binding_affinity_nm': np.random.lognormal(2, 1, 30),
        'pKd': np.random.normal(6.5, 1, 30),
        'mol_weight': np.random.normal(350, 100, 30),
        'logp': np.random.normal(2.5, 1.5, 30)
    })
    
    print("Created placeholder dataset")
    df.head()

In [ ]:
# Basic statistics
print("📊 Dataset Statistics")
print("=" * 25)
print(f"Total records: {len(df)}")
print(f"Unique proteins: {df['protein_name'].nunique()}")
print(f"Binding affinity range: {df['binding_affinity_nm'].min():.1f} - {df['binding_affinity_nm'].max():.1f} nM")
print(f"pKd range: {df['pKd'].min():.2f} - {df['pKd'].max():.2f}")

# Check for missing values
print("\n🔍 Missing Values:")
print(df.isnull().sum())

## Data Visualization

In [ ]:
# Create visualization plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Binding affinity distribution
axes[0,0].hist(df['binding_affinity_nm'], bins=20, alpha=0.7, color='skyblue')
axes[0,0].set_xlabel('Binding Affinity (nM)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_title('Distribution of Binding Affinities')
axes[0,0].set_yscale('log')

# 2. pKd distribution
axes[0,1].hist(df['pKd'], bins=20, alpha=0.7, color='lightcoral')
axes[0,1].set_xlabel('pKd')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title('Distribution of pKd Values')

# 3. Protein distribution
protein_counts = df['protein_name'].value_counts()
axes[1,0].bar(range(len(protein_counts)), protein_counts.values, color='lightgreen')
axes[1,0].set_xticks(range(len(protein_counts)))
axes[1,0].set_xticklabels(protein_counts.index, rotation=45, ha='right')
axes[1,0].set_ylabel('Count')
axes[1,0].set_title('Records per Protein')

# 4. Molecular weight vs binding affinity
if 'mol_weight' in df.columns:
    scatter = axes[1,1].scatter(df['mol_weight'], df['binding_affinity_nm'], 
                              c=df['pKd'], cmap='viridis', alpha=0.6)
    axes[1,1].set_xlabel('Molecular Weight (Da)')
    axes[1,1].set_ylabel('Binding Affinity (nM)')
    axes[1,1].set_title('Molecular Weight vs Binding Affinity')
    axes[1,1].set_yscale('log')
    plt.colorbar(scatter, ax=axes[1,1], label='pKd')

plt.tight_layout()
plt.show()

## Molecular Descriptor Analysis

In [ ]:
# Analyze molecular descriptors
try:
    from data_processing.molecular_processor import LigandProcessor
    
    processor = LigandProcessor()
    
    # Calculate descriptors for first few SMILES
    sample_smiles = df['ligand_smiles'].head(5).tolist()
    
    descriptor_data = []
    for smiles in sample_smiles:
        descriptors = processor.smiles_to_descriptors(smiles)
        descriptors['smiles'] = smiles
        descriptor_data.append(descriptors)
    
    descriptor_df = pd.DataFrame(descriptor_data)
    print("🧪 Molecular Descriptors (Sample):")
    print(descriptor_df.round(3))
    
except ImportError:
    print("Molecular processor not available - skipping descriptor analysis")

## Correlation Analysis

In [ ]:
# Correlation analysis
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f')
plt.title('Correlation Matrix of Molecular Properties')
plt.tight_layout()
plt.show()

# Show strongest correlations with binding affinity
if 'binding_affinity_nm' in correlation_matrix.columns:
    affinity_corr = correlation_matrix['binding_affinity_nm'].abs().sort_values(ascending=False)
    print("\n🔍 Strongest correlations with binding affinity:")
    print(affinity_corr.head(10))

## Machine Learning Model Training

In [ ]:
# Prepare data for machine learning
try:
    from data_processing.pipeline import DataPipeline
    from models.traditional_models import BindingAffinityPredictor
    
    # Initialize pipeline
    pipeline = DataPipeline()
    
    # Preprocess data
    processed_df = pipeline.preprocess_binding_data(df)
    
    # Prepare features and target
    X, y = pipeline.prepare_features_and_target(processed_df)
    
    print(f"Features shape: {X.shape}")
    print(f"Target shape: {y.shape}")
    print(f"Feature columns: {list(X.columns)}")
    
except ImportError as e:
    print(f"ML modules not available: {e}")
    print("Creating simple feature matrix...")
    
    # Simple feature preparation
    feature_cols = [col for col in df.columns if col in ['mol_weight', 'logp'] and col in df.columns]
    if feature_cols:
        X = df[feature_cols].fillna(0)
        y = df['pKd'] if 'pKd' in df.columns else df['binding_affinity_nm']
        print(f"Simple features: {feature_cols}")
    else:
        print("No suitable features found")

In [ ]:
# Train a simple model
if 'X' in locals() and 'y' in locals() and len(X) > 0:
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import r2_score, mean_squared_error
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Train Random Forest model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_train = rf_model.predict(X_train)
    y_pred_test = rf_model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    print("🤖 Random Forest Model Results:")
    print(f"Training R²: {train_r2:.3f}")
    print(f"Test R²: {test_r2:.3f}")
    print(f"Training RMSE: {train_rmse:.3f}")
    print(f"Test RMSE: {test_rmse:.3f}")
    
    # Feature importance
    if hasattr(rf_model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\n📊 Feature Importance:")
        print(importance_df)
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        plt.barh(importance_df['feature'], importance_df['importance'])
        plt.xlabel('Importance')
        plt.title('Feature Importance in Random Forest Model')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
    
else:
    print("No data available for model training")

## Prediction Visualization

In [ ]:
# Visualize predictions
if 'y_test' in locals() and 'y_pred_test' in locals():
    plt.figure(figsize=(10, 8))
    
    # Scatter plot of predictions vs actual
    plt.scatter(y_test, y_pred_test, alpha=0.6, color='blue', label='Predictions')
    
    # Perfect prediction line
    min_val = min(y_test.min(), y_pred_test.min())
    max_val = max(y_test.max(), y_pred_test.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', 
             label='Perfect Prediction', linewidth=2)
    
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Predicted vs Actual Values (R² = {test_r2:.3f})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add text with metrics
    plt.text(0.05, 0.95, f'R² = {test_r2:.3f}\nRMSE = {test_rmse:.3f}', 
             transform=plt.gca().transAxes, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
             verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    # Residuals plot
    residuals = y_test - y_pred_test
    
    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred_test, residuals, alpha=0.6, color='green')
    plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residuals Plot')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    print("No predictions available for visualization")

## Summary and Conclusions

In [ ]:
print("🎯 Analysis Summary")
print("=" * 25)
print(f"Dataset size: {len(df)} records")
print(f"Number of unique proteins: {df['protein_name'].nunique()}")
print(f"Binding affinity range: {df['binding_affinity_nm'].min():.1f} - {df['binding_affinity_nm'].max():.1f} nM")

if 'test_r2' in locals():
    print(f"\n🤖 Model Performance:")
    print(f"Test R²: {test_r2:.3f}")
    print(f"Test RMSE: {test_rmse:.3f}")
    
    if test_r2 > 0.7:
        print("✅ Model achieves target performance (R² > 0.7)")
    else:
        print("⚠️ Model performance below target - consider feature engineering")

print("\n📝 Next Steps:")
print("1. Collect more diverse training data")
print("2. Implement advanced feature engineering")
print("3. Try ensemble models and hyperparameter tuning")
print("4. Validate on external test sets")
print("5. Deploy model in production application")

## Interactive Predictions

In [ ]:
# Interactive prediction function
def predict_binding_affinity(mol_weight=350, logp=2.5):
    """Make a prediction for given molecular properties."""
    if 'rf_model' in locals() and hasattr(rf_model, 'predict'):
        # Create feature vector
        features = np.array([[mol_weight, logp]])
        
        # Make prediction
        prediction = rf_model.predict(features)[0]
        
        print(f"Input: MW = {mol_weight:.1f} Da, LogP = {logp:.2f}")
        print(f"Predicted pKd: {prediction:.2f}")
        print(f"Predicted binding affinity: {10**(-prediction) * 1e9:.1f} nM")
        
        return prediction
    else:
        print("No trained model available")
        return None

# Example predictions
print("🧪 Example Predictions:")
print("-" * 25)

examples = [
    (300, 2.0),   # Small, hydrophilic
    (400, 3.5),   # Medium, moderate lipophilicity  
    (500, 5.0)    # Large, lipophilic
]

for mw, lp in examples:
    predict_binding_affinity(mw, lp)
    print()